# Interaction Energies and Potential Energy Surfaces

Electronic Structure Theory (QM) can be used to compute the total energy of a molecule. This energy is defined as relative to
all nuclei and electrons being infinitely far apart.

This total energy is not particularly meaningful by itself, however *relative* energies are useful. Among the more useful things you can
do is compute the **interaction energy** between two (or more) molecules.

For this lab, we will focus on the interaction energy of two water molecules (a dimer). To compute the interaction energy $IE$,
we first compute the total energy of the dimer $E_{AB}$, then subtract the energy of each of the monomers $E_A$ and $E_B$.

$$ IE = E_{AB} - (E_A + E_B) $$

We will then compute the interaction energy varying the distance between the two water molecules, and then plot that surface.

## Computing energies with psi4

Psi4 (https://psicode.org) is an open-source package for running QM computations, written in C++ but with extensive python bindings.
First, we will import psi4, and a few other helpful packages

In [ ]:
import psi4
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Set the amount of memory psi4 will use in calculations
# 1 GB should be enough for this lab
psi4.set_memory('1 GB')

To compute the energy of a molecule, we need to

1. Define our molecule with `psi4.geometry`
1. Set the output file (optional, but if not done, output will be sent to this notebook)
1. Call `psi4.energy`, passing the level of theory and molecule

### Defining a molecule

One way to define a molecule is with the XYZ format. In this format, each line contains the atomic symbol, followed by the x, y, and z coordinates. The last line here will define our units.

In [ ]:
water = psi4.geometry(
        f"""
        O                     1.548014347149      0.000000000000     0.060071441686
        H                     0.548241218221      0.000000000000     0.038771425885
        H                     1.819295099755      0.000000000000    -0.902428819751
        units angstrom
        """   
        )

### Setting the output

In [ ]:
psi4.set_output_file("water_energy.out")

### Run the computation

The first argument to `psi4.energy` is the level of theory, then we can pass in the molecule.

In [ ]:
energy = psi4.energy('hf/sto-3g', mol=water)
print(f"The energy of water is {energy} hartrees at the hf/sto-3g level of theory")

The typical unit of energy is the `hartree`. This, and the `bohr`, are part of a system of units called `atomic units`. `1 hartree` is approximately `627.509 kcal/mol`

### Exercise

Modify the water molecule and recompute the energy. Does the energy increase or decrease?

## Computing the interaction energy

Below is the coordinates for a water dimer in units of **angstroms**. The first 3 lines are one molecule, then the last 3 lines are the second.

```
        O                     1.548014347149    0.000000000000     0.060071441686
        H                     0.548241218221    0.000000000000     0.038771425885
        H                     1.819295099755    0.000000000000    -0.902428819751
        O                    -2.000000000000    0.000000000000    -0.003828605717
        H                    -2.499886564464    0.866025403784    -0.014478613618
        H                    -2.499886564464   -0.866025403784    -0.014478613618
```

### Exercise

1. Compute the interaction energy of this water dimer at the `hf/sto-3g` level of theory. You will need 3 computations (one for the dimer, then one for each monomer)
2. Compute at the following levels of theory, and compare their energies:
    1. `b3lyp/sto-3g`
    1. `b3lyp/def2/svp`

**HINT**: Write a function that takes the level of theory as an argument, runs the three computations, then returns the computed interaction energy


## Varying the distance between water molecules

Next, we would like to vary the distance between the two water molecules. To do this, we can create a template of an molecule coordinates block, which we will substitute in values.
We will use python 'fstrings', which will automatically substitute in values and evaluate some expressions.

We are moving one of the water molecules along the X coordinate, so we add our desired distance to the x coordinate of one of the molecules. The easiest way to do this is with a function

![Water Dimer 1](images/water_dimer_1.png)


![Water Dimer 2](images/water_dimer_2.png)

In [ ]:
def create_dimer(distance):
    return psi4.geometry(
        f"""
        O                     1.548014347149             0.000000000000     0.060071441686
        H                     0.548241218221             0.000000000000     0.038771425885
        H                     1.819295099755             0.000000000000    -0.902428819751
        O                    {distance}                  0.000000000000    -0.003828605717
        H                    {distance-0.499886564464}   0.866025403784    -0.014478613618
        H                    {distance-0.499886564464}  -0.866025403784    -0.014478613618
        units angstrom
        """
    )

### Excercise

Create a function that returns the two monomers that will be needed to calculate the interaction energies

### Exercise: Creating the Potential Energy Surface

Now with functions for the monomers and dimers, we can run energy calculations along various distances

#### Outline

1. Write a function that computes the interaction energy at a given distance. The distance and the level of theory should be parameters to the function.
2. Use the numpy linspace function to create the distances you are going to compute. For this molecule, **go from -5.0 to -0.5 with 30 points**.
3. Compute using the `hf/sto-3g` level of theory.
4. Append the energies to a list, and then make a plot with matplotlib showing distance vs. interaction energy

**BONUS**: Plot with additional levels of theory:
1. `b3lyp/def2-svp`
2. `b3lyp/aug-cc-pvdz`
3. `mp2/sto-3g`